In [1]:
# Importing the libraries
import numpy as np 
import pandas as pd 

### Loading the dataset

In [2]:
df = pd.read_csv('../input/fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
# Shape of the data
df.shape

(20800, 5)

In [4]:
# Creating the feature matrix
X = df.drop('label', axis = 1)
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [5]:
# Get the dependent variable
y = df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [6]:
# Removing the NaN values
df = df.dropna()

In [7]:
# Shape of the data after removing Nan values 
df.shape

(18285, 5)

In [39]:
# Importing the essential libraries
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [9]:
# Vocabulary size
vocab_size = 5000

### One-Hot Representation

In [10]:
messages = df.copy()

In [11]:
# Resetting the indices after the removal of NaN values
messages.reset_index(inplace = True)

In [12]:
# Example text from our data 
print(messages['title'][5])
print(messages['text'][5])

Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (Exclusive Video) - Breitbart
In these trying times, Jackie Mason is the Voice of Reason. [In this week’s exclusive clip for Breitbart News, Jackie discusses the looming threat of North Korea, and explains how President Donald Trump could win the support of the Hollywood left if the U. S. needs to strike first.  “If he decides to bomb them, the whole country will be behind him, because everybody will realize he had no choice and that was the only thing to do,” Jackie says. “Except the Hollywood left. They’ll get nauseous. ” “[Trump] could win the left over, they’ll fall in love with him in a minute. If he bombed them for a better reason,” Jackie explains. “Like if they have no transgender toilets. ” Jackie also says it’s no surprise that Hollywood celebrities didn’t support Trump’s strike on a Syrian airfield this month. “They were infuriated,” he says. “Because it might only save lives. That

*Data Pre-processing*

In [13]:
# Again importing the essential libraries
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [14]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [15]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [16]:
onehot_rep = [one_hot(words,vocab_size) for words in corpus]
onehot_rep

[[406, 4603, 3384, 4961, 2821, 1840, 4994, 4144, 3301, 2314],
 [1831, 1947, 4850, 4795, 4396, 1508, 4971],
 [2751, 3509, 1058, 4310],
 [1308, 3033, 3591, 95, 1576, 3754],
 [4933, 4396, 4495, 453, 2074, 3248, 4396, 81, 1798, 1546],
 [3564,
  2678,
  2338,
  3576,
  4,
  4682,
  2159,
  3896,
  826,
  1078,
  4030,
  2656,
  505,
  4264,
  4971],
 [4121, 4234, 2059, 4608, 109, 3289, 4846, 1148, 4759, 2441, 2812],
 [4550, 4634, 831, 3298, 3980, 1513, 4682, 437, 4759, 2441, 2812],
 [1883, 3029, 2391, 1102, 758, 1603, 4858, 2868, 4682, 2823],
 [2967, 1946, 1775, 4953, 2217, 4030, 1525, 1363],
 [588, 4082, 3536, 545, 1262, 4729, 4729, 4479, 4323, 3453, 2208],
 [95, 2975, 2821, 1603, 4682, 3980],
 [2732, 2204, 572, 2796, 3804, 1998, 1863, 4372, 4767],
 [696, 2387, 2873, 4709, 1376, 470, 2682, 4759, 2441, 2812],
 [123, 2838, 2240, 3167, 3828, 4759, 2441, 2812],
 [1544, 4925, 1537, 4928, 1235, 1954, 3568, 1625, 2412, 4883],
 [2334, 4573, 1947],
 [186, 2471, 1026, 1183, 4682, 4482, 2913, 4971],


### Embedding Representation

In [17]:
sent_len = 20
embedded = pad_sequences(onehot_rep, padding = 'pre', maxlen = sent_len)
print(embedded)

[[   0    0    0 ... 4144 3301 2314]
 [   0    0    0 ... 4396 1508 4971]
 [   0    0    0 ... 3509 1058 4310]
 ...
 [   0    0    0 ... 4759 2441 2812]
 [   0    0    0 ... 4443  640 3467]
 [   0    0    0 ... 2242 2015 2216]]


In [18]:
len(embedded)

18285

In [19]:
embedded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  406,
       4603, 3384, 4961, 2821, 1840, 4994, 4144, 3301, 2314], dtype=int32)

### Creating the Model

***LSTM***

In [20]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = sent_len))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
y = messages['label']

In [22]:
import numpy as np
X_final = np.array(embedded)
y_final = np.array(y)

In [23]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [24]:
# Train-Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=30)

In [25]:
# Train the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30,batch_size=64)

Epoch 1/30
192/192 [==============================] - 2s 11ms/step - loss: 0.3366 - accuracy: 0.8380 - val_loss: 0.1900 - val_accuracy: 0.9198
Epoch 2/30
192/192 [==============================] - 2s 8ms/step - loss: 0.1422 - accuracy: 0.9429 - val_loss: 0.1940 - val_accuracy: 0.9152
Epoch 3/30
192/192 [==============================] - 2s 9ms/step - loss: 0.0995 - accuracy: 0.9619 - val_loss: 0.2011 - val_accuracy: 0.9183
Epoch 4/30
192/192 [==============================] - 2s 8ms/step - loss: 0.0717 - accuracy: 0.9758 - val_loss: 0.2282 - val_accuracy: 0.9157
Epoch 5/30
192/192 [==============================] - 2s 8ms/step - loss: 0.0531 - accuracy: 0.9811 - val_loss: 0.2725 - val_accuracy: 0.9176
Epoch 6/30
192/192 [==============================] - 2s 8ms/step - loss: 0.0340 - accuracy: 0.9887 - val_loss: 0.2810 - val_accuracy: 0.9155
Epoch 7/30
192/192 [==============================] - 2s 10ms/step - loss: 0.0238 - accuracy: 0.9925 - val_loss: 0.2866 - val_accuracy: 0.9201
Epoc

### Performance Metrics

In [26]:
y_pred = (model.predict_classes(X_test))

In [27]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
confusion_matrix(y_test,y_pred)
accuracy_score(y_test,y_pred)

0.9144987572493787

### Adding Dropout

In [28]:
from tensorflow.keras.layers import Dropout
# Creating the model
embedding_vector_features = 50
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = sent_len))
#model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 50)            250000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
X_final = np.array(embedded)
y_final = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=2)

# Train the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 2s 8ms/step - loss: 0.0249 - accuracy: 0.9918 - val_loss: 0.0502 - val_accuracy: 0.9865
Epoch 2/10
200/200 [==============================] - 2s 8ms/step - loss: 0.0059 - accuracy: 0.9987 - val_loss: 0.0625 - val_accuracy: 0.9843
Epoch 3/10
200/200 [==============================] - 2s 8ms/step - loss: 8.5614e-04 - accuracy: 0.9998 - val_loss: 0.0713 - val_accuracy: 0.9847
Epoch 4/10
200/200 [==============================] - 2s 8ms/step - loss: 1.2453e-04 - accuracy: 1.0000 - val_loss: 0.0788 - val_accuracy: 0.9836
Epoch 5/10
200/200 [==============================] - 2s 9ms/step - loss: 6.3954e-05 - accuracy: 1.0000 - val_loss: 0.0825 - val_accuracy: 0.9832
Epoch 6/10
200/200 [==============================] - 2s 11ms/step - loss: 4.4237e-05 - accuracy: 1.0000 - val_loss: 0.0863 - val_accuracy: 0.9832
Epoch 7/10
200/200 [==============================] - 2s 9ms/step - loss: 2.7925e-05 - accuracy: 1.0000 - val_loss: 0.0886 - val_ac

In [35]:
y_pred = (model.predict_classes(X_test))
print('\n Confusion Matrix:- \n',confusion_matrix(y_test,y_pred))
print('\n Accuracy:',accuracy_score(y_test,y_pred)* 100,'%')


 Confusion Matrix:- 
 [[3061   47]
 [  45 2333]]

 Accuracy: 98.32300401020781 %


***Bi-Directional LSTM***

In [40]:
# Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(vocab_size, embedding_vector_features, input_length=sent_len))
model1.add(Bidirectional(LSTM(100)))
# Adding the dropout rate of 0.3
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [41]:
len(embedded),y.shape

(18285, (18285,))

In [47]:
X_final=np.array(embedded)
y_final=np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state = 4)

model1.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0032 - accuracy: 0.9996 - val_loss: 0.2829 - val_accuracy: 0.9513
Epoch 2/10
200/200 [==============================] - 2s 12ms/step - loss: 0.0025 - accuracy: 0.9995 - val_loss: 0.2913 - val_accuracy: 0.9497
Epoch 3/10
200/200 [==============================] - 2s 11ms/step - loss: 0.0021 - accuracy: 0.9998 - val_loss: 0.3075 - val_accuracy: 0.9526
Epoch 4/10
200/200 [==============================] - 2s 11ms/step - loss: 0.0017 - accuracy: 0.9998 - val_loss: 0.3197 - val_accuracy: 0.9506
Epoch 5/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0017 - accuracy: 0.9995 - val_loss: 0.3591 - val_accuracy: 0.9490
Epoch 6/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0098 - accuracy: 0.9972 - val_loss: 0.2339 - val_accuracy: 0.9493
Epoch 7/10
200/200 [==============================] - 2s 11ms/step - loss: 0.0126 - accuracy: 0.9962 - val_loss: 0.2952 - val_accuracy: 0.9480

In [46]:
y_pred = (model.predict_classes(X_test))
print('\n Confusion Matrix:- \n',confusion_matrix(y_test,y_pred))
print('\n Accuracy:',accuracy_score(y_test,y_pred)* 100,'%')


 Confusion Matrix:- 
 [[3061   47]
 [  45 2333]]

 Accuracy: 98.32300401020781 %
